In [8]:
%reload_ext autoreload
%autoreload 2

import os
import argparse
import sys
import time

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *
from metadata import *
from data_manager import *

import numpy as np

from learning_utilities import *
from multiprocess import Pool

In [2]:
stack = 'MD635'
first_sec, last_sec = metadata_cache['section_limits'][stack]

In [3]:
setting = 10

In [12]:
if setting == 12:
    
    setting_nissl = 2
    setting_ntb = 10
    
    clf_nissl_allClasses = DataManager.load_classifiers(setting=setting_nissl)
    clf_ntb_allClasses = DataManager.load_classifiers(setting=setting_ntb)

else:
    clf_allClasses = DataManager.load_classifiers(setting=setting)

Setting 10: No classifier found for Amb.
Setting 10: No classifier found for LRt.
Setting 10: No classifier found for RMC.
Setting 10: No classifier found for 3N.
Setting 10: No classifier found for 4N.
Setting 10: No classifier found for Sp5I.
Setting 10: No classifier found for Sp5O.
Setting 10: No classifier found for PBG.
Setting 10: No classifier found for sp5.
Setting 10: No classifier found for outerContour.


In [13]:
def clf_predict(stack, sec):

    try:
        features = DataManager.load_dnn_features(stack=stack, section=sec)
    except Exception as e:
        sys.stderr.write('%s\n' % e.message)
        return

    if setting == 12:
        stain = 'nissl' # use some heuristic to decide the stain of current section

        if stain == 'nissl':
            setting_ = setting_nissl
            clf_allClasses_ = clf_nissl_allClasses
        else:
            setting_ = setting_ntb
            clf_allClasses_ = clf_ntb_allClasses
    else:
        setting_ = setting
        clf_allClasses_ = clf_allClasses

    for structure, clf in clf_allClasses_.iteritems():

        probs = clf.predict_proba(features)[:, clf.classes_.tolist().index(1.)]

        output_fn = DataManager.get_sparse_scores_filepath(stack=stack, structure=structure, setting=setting_, sec=sec)
        create_if_not_exists(os.path.dirname(output_fn))

        bp.pack_ndarray_file(probs, output_fn)

In [14]:
clf_predict(stack=stack, sec=202)

In [ ]:
t = time.time()

pool = Pool(8)
pool.map(lambda sec: clf_predict(stack=stack, sec=sec), range(200, 201))
# pool.map(lambda sec: clf_predict(stack=stack, sec=sec), range(first_sec, last_sec+1))
pool.close()
pool.join()

sys.stderr.write('Classifier predict: %.2f\n' % (time.time()-t)) # 30s each section, single process; 